In [1]:
#Install required libraries
!pip install keras==2.8
!pip install h5py
!pip install -U keras-tuner

In [2]:
#import for hdf5 table reading
import h5py
import numpy as np
import pandas as pd
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
import os
import time
LOG_DIR = f"{int(time.time())}"

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
#import tensorflow
import tensorflow as tf
tf.__version__

'2.8.0'

In [4]:
#import keras and Sequential
from tensorflow import keras 
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, RMSprop
keras.__version__

'2.8.0'

In [5]:
#Next 3 cells of code populate the backgroundVar and higgsVar table with desired data for background pattern
background = pd.HDFStore('background.hdf5', mode = 'r')
higgs = pd.HDFStore('higgs.hdf5', mode = 'r')

In [6]:
#background.keys()

In [7]:
#background['df'].keys()

In [8]:
#Within the key df there are many different keys, only actually prints higgs['df'].keys() with this arrangement
#background.keys()
#background['df'].keys()
#higgs.keys()
#higgs['df'].keys()

In [9]:
#Notes, could not find mJ1 and there were double dRB1Ph listed in README
backgroundVar = background['df'][['mJJ','cosThetaC','dRB1J1', 'cenPhJJ', 'pTJJ', 'pTBal', 'dEtaJJ', 'dRB1Ph', 'dPhiBBJJ', 'nJets', 'zep','etaJ5']]
higgsVar = higgs['df'][['mJJ','cosThetaC','dRB1J1', 'cenPhJJ', 'pTJJ', 'pTBal', 'dEtaJJ', 'dRB1Ph', 'dPhiBBJJ', 'nJets', 'zep','etaJ5']]

In [10]:
backgroundVar['isBackground'] = 1
higgsVar['isBackground'] = 0

In [11]:
#Table containing backgroundVariables
#backgroundVar

In [12]:
#Table containing higgsVariables
#higgsVar

In [13]:
#higgsVar.shape

In [14]:
backgroundValid, backgroundTrain = backgroundVar[:10000], backgroundVar[10000:] 
higgsValid, higgsTrain = higgsVar[:10000], higgsVar[10000:]
backgroundTrain = backgroundTrain[:10000]
higgsTrain = higgsTrain[:10000]

In [15]:
#Shape of all variables grabbed above
#backgroundValid.shape

In [16]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [17]:
train_labels = pd.concat([backgroundTrain['isBackground'],higgsTrain['isBackground']])
valid_labels = pd.concat([backgroundValid['isBackground'],higgsValid['isBackground']])
#train_labels

In [18]:
train_inputs = pd.concat([backgroundTrain[['mJJ','cosThetaC','dRB1J1', 'cenPhJJ', 'pTJJ', 'pTBal', 'dEtaJJ', 'dRB1Ph', 'dPhiBBJJ', 'nJets', 'zep','etaJ5']],higgsTrain[['mJJ','cosThetaC','dRB1J1', 'cenPhJJ', 'pTJJ', 'pTBal', 'dEtaJJ', 'dRB1Ph', 'dPhiBBJJ', 'nJets', 'zep','etaJ5']]])
valid_inputs = pd.concat([backgroundValid[['mJJ','cosThetaC','dRB1J1', 'cenPhJJ', 'pTJJ', 'pTBal', 'dEtaJJ', 'dRB1Ph', 'dPhiBBJJ', 'nJets', 'zep','etaJ5']],higgsValid[['mJJ','cosThetaC','dRB1J1', 'cenPhJJ', 'pTJJ', 'pTBal', 'dEtaJJ', 'dRB1Ph', 'dPhiBBJJ', 'nJets', 'zep','etaJ5']]])
#train_inputs

In [ ]:
def build_model(hp):
    model = keras.models.Sequential()
    
    model.add(Dense(12,activation = "relu", input_shape = (12,)))
    model.add(BatchNormalization())
    
    #layers
    for i in range(hp.Int('n_layers', 1, 6)):
        model.add(Dense(hp.Int(f'conv_{i}_units', min_value = 12, max_value = 288, step = 12)))
        model.add(Activation('relu'))
        model.add(Dropout(0.2))
    
    #final layer
    model.add(Dense(1, activation="sigmoid", name = "predictions"))
    
    #compile
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=["accuracy"])
    return model

tuner = RandomSearch(
    build_model,
    objective = "accuracy",
    max_trials = 100,
    executions_per_trial = 2,
    directory = LOG_DIR)

tuner.search(x=train_inputs,
             y=train_labels,
             verbose=2, # just slapping this here bc jupyter notebook. The console out was getting messy.
             epochs=1,
             batch_size=32,
             #callbacks=[tensorboard],  # if you have callbacks like tensorboard, they go here.
             validation_split=0.1
            )

Trial 5 Complete [00h 00m 14s]
accuracy: 0.4448888897895813

Best accuracy So Far: 0.44591666758060455
Total elapsed time: 00h 01m 15s

Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
5                 |5                 |n_layers
168               |96                |conv_0_units
216               |12                |conv_1_units
36                |12                |conv_2_units
288               |12                |conv_3_units
276               |12                |conv_4_units



In [ ]:
tuner.results_summary()

In [ ]:
model.layers
model.summary()

In [ ]:
from matplotlib import pyplot as plt
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

In [ ]:
#Functional Model example
#inp1 = Input((13, ))
#inp2 = Input((13, ))
#x1 = Flatten()(inp1)
#x2 = Flatten()(inp2)
#x = keras.layers.concatenate([x1, x2])
#x = keras.layers.Dense(32)(x)
#out = keras.layers.Dense(1, activation='sigmoid')(x)
#func = Model([inp1, inp2], out)
#func.compile(loss="binary_crossentropy",
#              optimizer="sgd",
#              metrics=["accuracy"])

In [ ]:
#Links

#some solutions to no loss decrease
#https://stackoverflow.com/questions/51113328/keras-high-loss-not-decreasing-with-each-epoch

#Tutorial on building networks
#https://www.youtube.com/watch?v=qFJeN9V1ZsI&ab_channel=freeCodeCamp.org

#Book on deep learning
#https://www.deeplearningbook.org/

In [ ]:
#([
#keras.layers.Flatten(input_shape=(12,)),
#keras.layers.Dense(12, activation="relu", input_shape=(12,)),
#keras.layers.Dense(30, activation="relu"),
#keras.layers.Dense(10, activation="relu"),
#keras.layers.Dense(1, activation="sigmoid", name="predictions")
#])